In [3]:
!pip install opencv-python


In [4]:
!pip install tensorflow_probability==0.8.0rc0 --user --upgrade

In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tensorflow_datasets

Note: you may need to restart the kernel to use updated packages.


#### Machine learning model which is trained to identify the letters


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset
ds_train, ds_test = tfds.load('emnist', split=['train', 'test'], shuffle_files=True, as_supervised=True)

# Preprocess the data
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, (28, 28, 1))
    image = tf.image.resize(image , (28, 28))
    return image, label

ds_train = ds_train.map(preprocess).batch(128)
ds_test = ds_test.map(preprocess).batch(128)

# Define the model architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(26, activation="softmax"),
    ]
)

ds_train = ds_train.map(lambda x, y: (x, tf.one_hot(y, depth=26)))
ds_test = ds_test.map(lambda x, y: (x, tf.one_hot(y, depth=26)))



# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

# Train the model
model.fit(ds_train, epochs=10, validation_data=ds_test)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(ds_test)
print("Test accuracy:", test_acc)

In [8]:
model.save(r'E:\New folder\my_model.h5')

#### Computer Vision code to detect mulitple letters

In [1]:
import cv2 as cv
import numpy as np
from tensorflow import keras

# Load the trained model

model = keras.models.load_model(r'E:\New folder\my_model.h5')

# Define a dictionary to map predicted labels to letters
letter_map = { 0: 'A',1: 'B',2: 'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'0',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z'
}

# Open the webcam
cap = cv.VideoCapture(0)

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    # Preprocess the frame (resize and convert to grayscale)

    if not ret:
        print("Unable to read the frame")
    else:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        x = cv.resize(gray, (28, 28))
        
#         x = np.expand_dims(x, axis=-1)
#         x = np.expand_dims(x, axis=0)
#         x = x / 255.0
        
        
#         x = x.reshape((1, 28, 28, 1))
        
        
# Threshold the image to convert to binary
        _, thresh = cv.threshold(x, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)

# Find contours to extract individual letters
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# Sort contours from left to right
        contours = sorted(contours, key=lambda c: cv.boundingRect(c)[0])

# Predict letters using the trained model
        letters = []
        for contour in contours:
            # Extract letter image and preprocess
            x, y, w, h = cv.boundingRect(contour)
            letter_img = thresh[y:y+h, x:x+w]
            letter_img = cv.resize(letter_img, (28, 28))
            letter_img = np.reshape(letter_img, (1, 28, 28, 1))
            letter_img = letter_img.astype('float32') / 255.0

            # Predict letter using the model
            prediction = model.predict(letter_img)
            
            letter = letter_map[np.argmax(prediction)]
            letters.append(letter)
            
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv.putText(frame, letter, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Output predicted letters as a list
        print(letters)
          # Display the frame
        cv.imshow('frame', frame)

          # Check for the 'q' key to exit
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 24ms/step
['L', 'L', 'L', 'T', 'L', 'L', 'Z', 'L', 'L']
1/1 [==============================] - 0s 23ms/step
['L', 'L', 'L', 'T', 'L', 'L', 'L', 'Z', 'L', 'L']
1/1 [==============================] - 0s 24ms/step
['H', 'L', 'L', 'T', 'L', 'L', 'L', 'Z', 'L', 'L']
1/1 [==============================] - 0s 24ms/step
['J', 'L', 'L', 'T', 'L', 'L', 'L', 'L', 'Z', 'L', 'L']
1/1 [==============================] - 0s 24ms/step
['H', 'L', 'L', 'T', 'L', 'L', 'L', 'Z', 'L', 'L', 'L']
1/1 [==============================] - 0s 23ms/step
['L', 'L', 'L', 'T', 'L', 'L', 'Z', 'L', 'L', 'L']
1/1 [==============================] - 0s 27ms/step
['T', 'L', 'L', 'T', 'L', 'L', 'L', 'Z', 'L', 'L', 'L']
1/1 [==============================] - 0s 35ms/step
['L', 'L', 'L', 'T', 'L', 'L', 'L', 'Z', 'L', 'L', 'L']
1/1 [==============================] - 0s 25ms/step
['H', 'L', 'L', 'T', 'L', 'L', 'L', 'Z', 'L', 'L']
1/1 [==============================] - 0s 34ms/step
['L', 


KeyboardInterrupt



In [15]:
!git clone https://github.com/elephantrobotics/pymycobot.git
    

Cloning into 'pymycobot'...


#### Second machine learning model


In [5]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

# Load the dataset
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Preprocess the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

model1 = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model1.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model1.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

test_loss, test_acc = model1.evaluate(X_test, y_test)
print("Test Accuracy:", test_acc)


# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/20
1875/1875 [==============================] - 53s 28ms/step - loss: 0.1461 - accuracy: 0.9562 - val_loss: 0.0569 - val_accuracy: 0.9815
Epoch 2/20
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0517 - accuracy: 0.9839 - val_loss: 0.0464 - val_accuracy: 0.9855
Epoch 3/20
1875/1875 [==============================] - 45s 24ms/step - loss: 0.0314 - accuracy: 0.9900 - val_loss: 0.0434 - val_accuracy: 0.9857
Epoch 4/20
1875/1875 [==============================] - 44s 24ms/step - loss: 0.0202 - accuracy: 0.9936 - val_loss: 0.0453 - val_accuracy: 0.9863
Epoch 5/20
1875/1875 [==============================] - 55s 29ms/step - loss: 0.0148 - accuracy: 0.9950 - val_loss: 0.0411 - val_accuracy: 0.9873
Epoch 6/20
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0104 - accuracy: 0.9965 - val_loss: 0.0478 - val_accuracy: 0.9860
Epoch 7/20
1875/1875 [==============================] - 61s 32ms/step - loss: 0.0067 - accuracy: 0.9980 - val_loss: 0.0565 -

In [23]:
## saving the second machine learning model
model1.save(r'E:\New folder\my_model1.h5')


#### computer vision code to do letter detection using second machine learning model

In [7]:
# Find contours to extract individual letters
import cv2 as cv
import numpy as np
from tensorflow import keras

# Load the trained model

model1 = keras.models.load_model(r'E:\New folder\my_model1.h5')

# Define a dictionary to map predicted labels to letters
letter_map = { 0: 'A',1: 'B',2: 'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'0',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z'
}

# Open the webcam
cap = cv.VideoCapture(0)

while True:
    # Capture a frame from the webcam
    ret, frame = cap.read()
    # Preprocess the frame (resize and convert to grayscale)

    if not ret:
        print("Unable to read the frame")
    else:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)  
        x = cv.resize(gray, (28, 28))
        
#         x = np.expand_dims(x, axis=-1)
#         x = np.expand_dims(x, axis=0)
#         x = x / 255.0
#         x = x.reshape((1, 28, 28, 1))
# Threshold the image to convert to binary
        _, thresh = cv.threshold(x, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)

# Find contours to extract individual letters
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# Sort contours from left to right
        contours = sorted(contours, key=lambda c: cv.boundingRect(c)[0])

# Predict letters using the trained model
        letters = []
        for contour in contours:
            # Extract letter image and preprocess
            x, y, w, h = cv.boundingRect(contour)
            letter_img = thresh[y:y+h, x:x+w]
            letter_img = cv.resize(letter_img, (28, 28))
            letter_img = np.reshape(letter_img, (1, 28, 28, 1))
            letter_img = letter_img.astype('float32') / 255.0

            # Predict letter using the model
            prediction = model.predict(letter_img)
            
            letter = letter_map[np.argmax(prediction)]
            letters.append(letter)
            
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv.putText(frame, letter, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Output predicted letters as a list
        print(letters)


         

          # Display the frame
        cv.imshow('frame', frame)

          # Check for the 'q' key to exitq
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv.destroyAllWindows()


1/1 [==============================] - 0s 23ms/step
['L', 'L', 'L']
1/1 [==============================] - 0s 24ms/step
['L', 'L', 'L', 'L']
1/1 [==============================] - 0s 8ms/step
['L', 'L', 'L', 'L', 'L', 'L']
1/1 [==============================] - 0s 21ms/step
['L', 'L']
1/1 [==============================] - 0s 12ms/step
['L', 'L']
1/1 [==============================] - 0s 15ms/step
['L', 'L', 'L']
1/1 [==============================] - 0s 12ms/step
['L', 'L']
1/1 [==============================] - 0s 17ms/step
['L', 'L']
1/1 [==============================] - 0s 25ms/step
['L', 'L', 'L']
1/1 [==============================] - 0s 31ms/step
['L', 'L']
1/1 [==============================] - 0s 23ms/step
['L', 'L']
1/1 [==============================] - 0s 24ms/step
['L', 'L', 'L', 'L']
1/1 [==============================] - 0s 21ms/step
['L', 'L']
1/1 [==============================] - 0s 23ms/step
['L', 'L']
1/1 [==============================] - 0s 23ms/step
['L', 'L']
1

In [48]:
coords = mc.get_coords()
print(coords)

[204.0, 203.2, 119.7, -173.72, 4.96, -20.19]


In [25]:
list1 = [[114.8, -177.1, 65.2, 176.92, -6.85, -159.28],[74.3, -63.4, 407.8, -91.42, 10.22, -91.13],[173.6, 5.6, 78.9, -178.0, 7.72, -43.03],[74.3, -63.4, 407.8, -91.42, 10.22, -91.13],[118.6, -180.0, 57.6, 171.39, 2.24, -102.59],[74.3, -63.4, 407.8, -91.42, 10.22, -91.13],[183.5, -53.8, 69.2, 176.34, 7.27, -90.89]]

In [26]:
for i in range(7):
    mc.send_coords(list1[i],20)
    time.sleep(10)
    if i%3 == 0:
        pump_on()
    else:
        pump_off()

pump_off()


SerialException: WriteFile failed (PermissionError(13, 'Access is denied.', None, 5))

In [75]:
list1 =[[110.5, -166.6, 83.8, 172.63, -2.11, -48.15],[46.2, -45.9, 407.2, 95.61, -66.15, 107.46],[163.0, 33.8, 58.3, -179.54, 0.98, 73.04],]

In [79]:
from pymycobot.mypalletizer import MyPalletizer
from pymycobot.genre import Coord
import time
#mc = MyPalletizer("COM3", 115200)

coords = mc.get_coords()
print(coords)


[5.2, -88.7, 228.7, 99.59, 45.86, 61.67]


In [179]:
set_tool_reference(coords)

NameError: name 'set_tool_reference' is not defined